1. Environment Setup

In [3]:
!pip install numpy scikit-learn torch torchvision

  Using cached torchvision-0.19.1-cp310-cp310-manylinux1_x86_64.whl.metadata (6.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 221.3 kB/s eta 0:00:00a 0:00:01


Importing required libraries

In [4]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


2. Data Loading

In [5]:
# Load the digits dataset
digits = load_digits()
X = digits.data
y = digits.target

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Logistic Regression Model

In [6]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=1000)

Model Accuracy, Model Size, Inference Time

In [7]:
from time import time

# Calculate accuracy
accuracy = model.score(X_test, y_test)
print(f"Model Accuracy: {accuracy:.4f}")

# Model size
import sys
model_size = sys.getsizeof(model.coef_) + sys.getsizeof(model.intercept_)
print(f"Model Size (bytes): {model_size}")

# Inference time
start_time = time()
predictions = model.predict(X_test)
inference_time = time() - start_time
print(f"Inference Time (seconds): {inference_time:.4f}")


Model Accuracy: 0.9722
Model Size (bytes): 240
Inference Time (seconds): 0.0007


Quantize the Model

In [8]:
def quantize_model(model, scale_factor=2**7):
    quantized_weights = model.coef_ / scale_factor
    quantized_intercept = model.intercept_ / scale_factor
    return quantized_weights, quantized_intercept


Inference Using Quantized Model

In [9]:
def quantized_inference(X, quantized_weights, quantized_intercept):
    quantized_predictions = np.dot(X, quantized_weights.T) + quantized_intercept
    return np.argmax(quantized_predictions, axis=1)


Report Quantized Model Accuracy, Size, Inference Time

In [10]:
# Quantize the model
quantized_weights, quantized_intercept = quantize_model(model)

# Quantized model accuracy
quantized_predictions = quantized_inference(X_test, quantized_weights, quantized_intercept)
quantized_accuracy = np.mean(quantized_predictions == y_test)
print(f"Quantized Model Accuracy: {quantized_accuracy:.4f}")

# Quantized model size
quantized_model_size = sys.getsizeof(quantized_weights) + sys.getsizeof(quantized_intercept)
print(f"Quantized Model Size (bytes): {quantized_model_size}")

# Quantized inference time
start_time = time()
quantized_predictions = quantized_inference(X_test, quantized_weights, quantized_intercept)
quantized_inference_time = time() - start_time
print(f"Quantized Inference Time (seconds): {quantized_inference_time:.4f}")


Quantized Model Accuracy: 0.9722
Quantized Model Size (bytes): 5440
Quantized Inference Time (seconds): 0.0007


Model Size Comparison

In [11]:
print("\nModel Size Comparison:")
print(f"Original Model Size: {model_size} bytes")
print(f"Quantized Model Size: {quantized_model_size} bytes")



Model Size Comparison:
Original Model Size: 240 bytes
Quantized Model Size: 5440 bytes
